In [1]:
from selenium import webdriver
from bs4 import BeautifulSoup
import re
import pandas as pd
import time
import sys

def PAGE_DOWN():
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(3)
    
keyword = input("인스타그램에서 원하는 검색어를 입력하세요: ")
#count_down = int(input('스크롤 다운 횟수를 입력하세요(정수입력):'))

driver = webdriver.Chrome("./chromedriver")
driver.get("https://www.instagram.com/")
time.sleep(3)
id_ = 'chaerin1295'
pw = 'greeny26'
# 로그인 
id_input = driver.find_element_by_css_selector('#loginForm > div > div:nth-child(1) > div > label > input')
id_input.send_keys(id_)
password_input = driver.find_element_by_css_selector('#loginForm > div > div:nth-child(2) > div > label > input')
password_input.send_keys(pw)
driver.find_element_by_css_selector("#loginForm > div > div:nth-child(3)").click()
time.sleep(3)
state = input('로그인이 잘 되었나요?[y/n]:')
if state == 'n' or state == 'N' or state == 'ㅜ':
    print('종료합니다.')
    driver.close()
    sys.exit()
else:
    pass

In [9]:
#driver.find_element_by_css_selector('body > div.RnEpo.Yx5HN > div > div > div > div.mt3GC > button.aOOlW.HoLwm').click()

In [2]:
# 검색결과 페이지로 이동
url = 'https://www.instagram.com/explore/tags/'
keyword='밥'
driver.get(url+keyword)
time.sleep(3)
body = driver.find_element_by_tag_name('body')

In [3]:
## 첫 번째 게시물 클릭 
first_post = driver.find_element_by_css_selector('#react-root > section > main > article > div.EZdmt > div > div > div:nth-child(1) > div:nth-child(1) > a > div > div._9AhH0')
first_post.click()

## 본문 내용
content = soup.select('div.C4VMK > span')[0].text except: content = ' '

## 해시태그
tags = re.findall(r'#[^\s#,\\]+', content) 

## 날짜
dtime = str(soup.select('time._1o9PC.Nzb55')[0]['datetime'])
date = dtime[:10]

In [4]:
#본문 내용 가져오기

def get_content(driver):

    html = driver.page_source
    soup = BeautifulSoup(html,'html.parser')
    
    #본문내용
    try:
        content = soup.select('div.C4VMK > span')[0].text
    except:
        content = ' '

    #날짜
    try:
        dtime = str(soup.select('time._1o9PC.Nzb55')[0]['datetime'])
        date = dtime[:10]
    except:
        date = ' '

    # 해시태그 
    try: 
        tags = re.findall(r'#[^\s#,\\]+', content) 
        
    except:
        tags=' '

    data = [date, tags]
    return data

#get_content(driver)

In [5]:
def next_page(driver):
    next_page = driver.find_element_by_css_selector('body > div._2dDPU.CkGkG > div.EfHg9 > div > div > a._65Bje.coreSpriteRightPaginationArrow')
    next_page.click()
    time.sleep(3)
    
#next_page(driver)

In [6]:
#크롤링 결과를담을 리스트 생성
result = []

# 여러 게시글 수집하기
target = 1200     # 크롤링할 게시글 수
for i in range(target):
    # 게시글 수집에 오류 발생시 5초 대기후, 다음 게시글로 넘어가도록 예외처리 구문 활용
    try: 
        try:
            data = get_content(driver)    # 게시글 정보 가져오기
            result.append(data)
            next_page(driver)
        except:
            driver.implicitly_wait(10)
            next_page(driver)

    except:
        break
        
    if i % 10 == 0:
        df = pd.DataFrame({'결과' : result})
        df.to_csv(f'data/insta_밥_{i}th.csv',encoding='utf-8-sig',index=False)

In [10]:
result[:2]

[['2021-08-04', []],
 ['2021-07-13', ['#아침식사', '#피마자', '#나물남편은']]]

In [19]:
df = pd.DataFrame(result)
df.columns = ['dates','tags']
df.to_csv('인스타해시태그_밥.csv', index=False, encoding='utf-8-sig')